# 1. テキストデータのTF-IDFベクトル化関数
問題:
複数の商品説明文（文字列のリスト）を受け取り、scikit-learn の TfidfVectorizer を使用して各文書をTF-IDFベクトルに変換する関数 vectorize_texts_tfidf(texts) を作成してください。
ベクトル化の際には、一般的な日本語のストップワード（例: 「の」「です」「ます」など。簡単なリストで可）を除外し、トークン化は単語単位（スペース区切り、または簡易的な形態素解析ライブラリ利用を想定するが、ここでは簡易的にスペース区切りで良い）で行うこととします。
戻り値は、TF-IDF行列（疎行列または密行列）と、使用したベクトライザのインスタンスとします。

期待される動作例:

```Python
from sklearn.feature_extraction.text import TfidfVectorizer

# 簡易的な日本語ストップワードリストの例
japanese_stopwords = ["の", "に", "は", "を", "た", "が", "で", "て", "と", "し", "れ", "さ", "ある", "いる", "も", "する", "から", "な", "こと", "として", "です", "ます"]

def vectorize_texts_tfidf(texts, stopwords=japanese_stopwords):
    # ここに処理を記述
    pass

descriptions = [
    "高機能 な スマートフォン です",
    "軽量 で 高性能 な ノートパソコン",
    "この スマートフォン は 最新 です"
]
tfidf_matrix, vectorizer = vectorize_texts_tfidf(descriptions)
# tfidf_matrix は (文書数 x 特徴語数) の行列
# vectorizer.get_feature_names_out() などで特徴語を確認できる
```
注: 実際にはjanomeやMeCabといった形態素解析ライブラリと組み合わせることが望ましいですが、テストの簡略化のため上記のような想定としています。


# 回答

# 2. 簡単な商品カテゴリ分類モデルの訓練と評価
問題:
商品名（文字列のリスト X_texts）とそれに対応するカテゴリラベル（文字列のリスト y_labels）が与えられます。
これらのデータを用いて、以下の手順で簡単な商品カテゴリ分類モデルを訓練し、評価する関数 train_and_evaluate_classifier(X_texts, y_labels) を作成してください。

TfidfVectorizer を用いて商品名をベクトル化します。
データを訓練用とテスト用に分割します（例: 80%訓練、20%テスト）。
scikit-learn の LogisticRegression （または NaiveBayes）を用いて分類モデルを訓練します。
テストデータでモデルの正解率（accuracy）を計算し、返します。
期待される動作例:

```Python
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # または from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

def train_and_evaluate_classifier(X_texts, y_labels):
    # ここに処理を記述
    pass

# サンプルデータ
X_texts = ["最新 iPhone 15 Pro", "美味しいリンゴ 青森産", "高性能ノートパソコン XYZ", "オーガニックコットン Tシャツ", "格安 Android スマホ"]
y_labels = ["スマートフォン", "食品", "PC", "衣類", "スマートフォン"]

accuracy = train_and_evaluate_classifier(X_texts, y_labels)
# accuracy は 0.0 から 1.0 の間の数値
```


# 回答

# 3. コサイン類似度計算関数
問題:
2つの特徴ベクトル（NumPy配列）vec1 と vec2 を受け取り、それらのコサイン類似度を計算する関数 calculate_cosine_similarity(vec1, vec2) を作成してください。
コサイン類似度は、ベクトルの内積をそれぞれのベクトルのL2ノルム（ユークリッドノルム）の積で割った値です。ゼロ除算が発生する場合は0を返すようにしてください。

期待される動作例:

```Python
import numpy as np

def calculate_cosine_similarity(vec1, vec2):
    # ここに処理を記述
    pass

vec_a = np.array([1, 1, 0, 1, 0])
vec_b = np.array([1, 0, 1, 1, 1])
vec_c = np.array([0, 0, 0, 0, 0])

similarity_ab = calculate_cosine_similarity(vec_a, vec_b) # 約0.577
similarity_ac = calculate_cosine_similarity(vec_a, vec_c) # 0.0
```


# 回答

# 4. K-Meansクラスタリングの適用とクラスタ中心の取得
問題:
商品データ（各商品が複数の数値特徴を持つと仮定し、NumPy配列 X_features で与えられる）とクラスタ数 k を受け取り、scikit-learn の KMeans を用いてデータをクラスタリングする関数 perform_kmeans_clustering(X_features, k) を作成してください。
この関数は、各データポイントが属するクラスタのラベル（配列）と、各クラスタの中心座標（配列）を返すものとします。

期待される動作例:

```Python
import numpy as np
from sklearn.cluster import KMeans

def perform_kmeans_clustering(X_features, k):
    # ここに処理を記述
    pass

# サンプルデータ (商品数 x 特徴数)
X_features = np.array([
    [1, 2], [1.5, 1.8], [5, 8],
    [8, 8], [1, 0.6], [9, 11]
])
k = 2
cluster_labels, cluster_centers = perform_kmeans_clustering(X_features, k)
# cluster_labels は各データ点がどのクラスタに属するかを示す配列 (例: [0, 0, 1, 1, 0, 1])
# cluster_centers は各クラスタの中心座標を示す配列 (例: [[1.16, 1.46], [7.33, 9.0]])
```


# 回答

# 5. 混同行列からの評価指標計算関数
問題:
ある二値分類モデルのテスト結果として得られた混同行列（2x2のNumPy配列）が与えられます。この混同行列から、正解率（Accuracy）、適合率（Precision）、再現率（Recall）、F1スコアを計算する関数 calculate_metrics_from_confusion_matrix(cm) を作成してください。
混同行列の形式は [[TN, FP], [FN, TP]] （TN: True Negative, FP: False Positive, FN: False Negative, TP: True Positive）とします。ゼロ除算が発生する場合は0を返すようにしてください。

期待される動作例:

```Python
import numpy as np

def calculate_metrics_from_confusion_matrix(cm):
    # ここに処理を記述
    pass

# 例: TN=50, FP=10, FN=5, TP=100
confusion_matrix = np.array([[50, 10], [5, 100]])
metrics = calculate_metrics_from_confusion_matrix(confusion_matrix)
# metrics は {'accuracy': accuracy_val, 'precision': precision_val, 'recall': recall_val, 'f1_score': f1_score_val} のような辞書
# 期待値: accuracy 約0.909, precision 約0.909, recall 約0.952, f1_score 約0.930
```


# 回答

# 6. 特徴量の標準化関数
問題:
数値特徴からなるデータセット（NumPy配列 X、各行がサンプル、各列が特徴量）を受け取り、各特徴量を標準化（平均0、標準偏差1にスケーリング）する関数 standardize_features(X) を作成してください。scikit-learn の StandardScaler を使用するか、NumPyで直接計算しても構いません。

期待される動作例:

```Python
import numpy as np
# from sklearn.preprocessing import StandardScaler # 使っても良い

def standardize_features(X):
    # ここに処理を記述
    pass

X_original = np.array([[1, -1, 2],
                       [2, 0, 0],
                       [0, 1, -1]], dtype=float)
X_standardized = standardize_features(X_original)
# X_standardized の各列は平均がほぼ0、標準偏差がほぼ1になる
# 例:
# [[ 0.         -1.22474487  1.33630621]
#  [ 1.22474487  0.         -0.26726124]
#  [-1.22474487  1.22474487 -1.06904497]]
```


# 回答

# 7. LightGBM/XGBoostの主要ハイパーパラメータ設定の理解
問題:
LightGBMまたはXGBoostのような勾配ブースティング木モデルにおいて、モデルの複雑さを制御し、過学習を抑制するために調整されることが多い主要なハイパーパラメータを3つ挙げ、それぞれのパラメータがモデルにどのような影響を与えるかを簡単に説明してください。そして、それらのパラメータを指定してモデルを初期化する（訓練は不要）簡単なPythonコードスニペットを lightgbm.LGBMClassifier または xgboost.XGBClassifier を用いて示してください。

期待される説明とコード例:

例として挙げるパラメータ:
n_estimators (木の数): 増やすとモデルの表現力は上がるが、過学習しやすくなる。
learning_rate (学習率): 小さいほど学習は慎重に進み、汎化性能が上がることがあるが、多くの木が必要になる。
max_depth (木の深さ): 深いほど複雑な関係を学習できるが、過学習しやすくなる。
（その他、num_leaves, min_child_samples, subsample, colsample_bytree なども候補）

```Python
# LightGBMの場合の例
import lightgbm as lgb

def initialize_lgbm_with_params(n_estimators, learning_rate, max_depth):
    model = lgb.LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42 # 再現性のため
    )
    return model

# 使用例
lgbm_model = initialize_lgbm_with_params(n_estimators=100, learning_rate=0.1, max_depth=5)
print(lgbm_model.get_params())
```

注: この問題はコーディングそのものより、ハイパーパラメータの知識を問うものです。

# 回答

# 8. 不均衡データに対する簡易オーバーサンプリング関数
問題:
二値分類タスクにおいて、少数派クラスのデータが極端に少ない不均衡データセット（特徴量 X とラベル y、NumPy配列）が与えられたとします。少数派クラスのサンプルを単純に複製することでオーバーサンプリングを行う関数 simple_oversample(X, y) を作成してください。多数派クラスのサンプル数は変更せず、少数派クラスのサンプル数が多数派クラスのサンプル数と同じになるように複製するものとします。
（imblearn ライブラリの RandomOverSampler のような高度なものではなく、基本的な動作を実装してください。）

期待される動作例:

```Python
import numpy as np
from collections import Counter

def simple_oversample(X, y):
    # ここに処理を記述
    pass

# サンプルデータ (特徴量は1次元で簡略化)
X_imbalanced = np.array([[1],[2],[3],[4],[5],[6],[7],[8],[9],[10]])
y_imbalanced = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0]) # クラス1が少数派

X_resampled, y_resampled = simple_oversample(X_imbalanced, y_imbalanced)
# Counter(y_resampled) の結果、両クラスのサンプル数が等しくなる (この例ではクラス0が8件、クラス1が2件なので、クラス1を6件複製し合計16件、各8件)
# X_resampled の形状と y_resampled の形状も確認
```


# 回答

# 9. 商品レビューの簡易感情分析（ポジネガ辞書利用）
問題:
商品レビューのテキストと、ポジティブ単語のセット positive_words、ネガティブ単語のセット negative_words を受け取り、レビューの感情スコアを計算する関数 calculate_sentiment_score(review_text, positive_words, negative_words) を作成してください。
スコアは「レビューに含まれるポジティブ単語の数 - レビューに含まれるネガティブ単語の数」とします。単語のカウントは単純な出現回数とし、大文字・小文字は区別しないものとします。

期待される動作例:

```Python
def calculate_sentiment_score(review_text, positive_words, negative_words):
    # ここに処理を記述
    pass

positive_set = {"良い", "素晴らしい", "満足", "最高", "便利"}
negative_set = {"悪い", "残念", "不満", "ひどい", "壊れた"}

review1 = "この商品はとても良い。素晴らしい！"
score1 = calculate_sentiment_score(review1, positive_set, negative_set) # 期待値: 2 (良い, 素晴らしい)

review2 = "期待外れで残念。少し悪い点もある。"
score2 = calculate_sentiment_score(review2, positive_set, negative_set) # 期待値: -2 (残念, 悪い)

review3 = "特にコメントなし。"
score3 = calculate_sentiment_score(review3, positive_set, negative_set) # 期待値: 0
```


# 回答

# 10. 時系列データからの移動平均計算関数
問題:
商品の売上データなどの時系列データ（数値のリストまたはNumPy配列 series）と、ウィンドウサイズ window_size を受け取り、移動平均を計算してリストまたはNumPy配列で返す関数 calculate_moving_average(series, window_size) を作成してください。
計算結果の配列の長さは、元の時系列データから window_size - 1 を引いたものになります（ウィンドウ内のデータが揃わない先頭部分は計算しない）。

期待される動作例:

```Python
import numpy as np

def calculate_moving_average(series, window_size):
    # ここに処理を記述
    pass

sales_data = np.array([10, 12, 11, 15, 16, 14, 18, 20, 19, 22])
window = 3
moving_avg = calculate_moving_average(sales_data, window)
# 期待される出力 (numpy array): [11.        , 12.66666667, 14.        , 15.        , 16.        , 17.33333333, 19.        , 20.33333333]
# ( (10+12+11)/3, (12+11+15)/3, ... )
```


# 回答